In [ ]:
from lerobot.common.robot_devices.motors.feetech import FeetechMotorsBus, FeetechMotorsBusConfig
import time

# Configuration du bus moteur
config = FeetechMotorsBusConfig(
    port="/dev/ttyACM1",  # ← vérifie que c'est bien le port
    motors={"gripper": [4, "sts3215"]}
)

# Création et connexion
bus = FeetechMotorsBus(config=config)
bus.connect()

# Assurer que le couple est désactivé pour mouvement manuel
bus.write("Torque_Enable", 0, motor_names=["gripper"])

# Lecture avant mouvement manuel
pos1 = bus.read("Present_Position", motor_names=["gripper"])
print(f"Position 1 : {pos1}")

input("→ Bouge manuellement le gripper (sans couple), puis appuie sur Entrée...")

# Lecture après mouvement manuel
pos2 = bus.read("Present_Position", motor_names=["gripper"])
print(f"Position 2 : {pos2}")

bus.disconnect()



Position 1 : [2189]
Position 2 : [2189]


In [13]:
from lerobot.common.robot_devices.motors.feetech import FeetechMotorsBusConfig, FeetechMotorsBus
from lerobot.common.robot_devices.robots.feetech_calibration import run_arm_auto_calibration
import json
import os

# Configuration du bus moteur
config = FeetechMotorsBusConfig(
    port="/dev/ttyACM0",  # ← vérifie que c'est bien le port du bras follower
    motors={
        "shoulder_pan": [1, "sts3215"],
        "shoulder_lift": [2, "sts3215"],
        "elbow_flex": [3, "sts3215"],
        "wrist_flex": [4, "sts3215"],
        "wrist_roll": [5, "sts3215"],
        "gripper": [6, "sts3215"],
    }
)


In [14]:

# Création et connexion au bus
bus = FeetechMotorsBus(config=config)
bus.connect()

# Désactive le couple pour autoriser la calibration
bus.write("Torque_Enable", 0)


In [6]:
from lerobot.common.robot_devices.motors.feetech import FeetechMotorsBusConfig, FeetechMotorsBus
from lerobot.common.robot_devices.robots.feetech_calibration import run_arm_auto_calibration_so100
import time
import os

# Configuration du bus moteur
config = FeetechMotorsBusConfig(
    port="/dev/ttyACM0",
    motors={
        "shoulder_pan": [1, "sts3215"],
        "shoulder_lift": [2, "sts3215"],
        "elbow_flex": [3, "sts3215"],
        "wrist_flex": [4, "sts3215"],
        "wrist_roll": [5, "sts3215"],
        "gripper": [6, "sts3215"],
    }
)

# Connexion au bus
bus = FeetechMotorsBus(config=config)
bus.connect()

try:
    print("↪️  Désactivation du couple pour la calibration")
    bus.write("Torque_Enable", 0)

    print("🚦 Début de la calibration avec ancienne logique (attention à wrist_flex)")
    calib_data = run_arm_auto_calibration_so100(
        arm=bus,
        robot_type="so100",
        arm_name="main",
        arm_type="follower"
    )

    print("\n✅ Calibration terminée (aucun fichier enregistré)")
    print("📊 Données de calibration (extrait) :")
    for name, val in zip(calib_data["motor_names"], calib_data["homing_offset"]):
        print(f"  - {name}: offset = {val:.1f}")

except Exception as e:
    print(f"\n💥 Une erreur est survenue : {e}")
finally:
    print("\n🔌 Déconnexion...")
    bus.disconnect()


↪️  Désactivation du couple pour la calibration
🚦 Début de la calibration avec ancienne logique (attention à wrist_flex)

Running calibration of so100 main follower...

Move arm to initial position
See: https://raw.githubusercontent.com/huggingface/lerobot/main/media/so100/follower_initial.webp
arm.read("Present_Position", "elbow_flex")=array([3064], dtype=int32)
Calibrate shoulder_pan
Calibrate gripper
Calibrate wrist_flex
Calibrate elbow_flex

🔌 Déconnexion...


KeyboardInterrupt: 

In [4]:
from lerobot.common.robot_devices.motors.feetech import FeetechMotorsBus, FeetechMotorsBusConfig

# Configuration du port et des moteurs
config = FeetechMotorsBusConfig(
    port="/dev/ttyACM0",  # Assure-toi que c'est le bon port !
    motors={
        "shoulder_pan": [1, "sts3215"],
        "shoulder_lift": [2, "sts3215"],
        "elbow_flex": [3, "sts3215"],
        "wrist_flex": [4, "sts3215"],
        "wrist_roll": [5, "sts3215"],
        "gripper": [6, "sts3215"],
    }
)

# Connexion au bus
bus = FeetechMotorsBus(config=config)
bus.connect()

In [5]:
# 💡 Désactivation du couple sur tous les moteurs
bus.write("Torque_Enable", 0)

print("✅ Couple désactivé, le bras peut être bougé manuellement.")

# Déconnexion propre
bus.disconnect()

✅ Couple désactivé, le bras peut être bougé manuellement.


In [41]:
from lerobot.common.robot_devices.motors.feetech import FeetechMotorsBusConfig, FeetechMotorsBus
from lerobot.common.robot_devices.robots.feetech_calibration import move_to_calibrate, apply_offset
import time

# Configuration pour tester uniquement wrist_flex
config = FeetechMotorsBusConfig(
    port="/dev/ttyACM0",
    motors={
        "wrist_flex": [4, "sts3215"],
    }
)

bus = FeetechMotorsBus(config=config)
bus.connect()

try:
    print("↪️ Désactivation du couple")
    bus.write("Torque_Enable", 0)

    print("✅ Couple OFF. Tu peux bouger le moteur manuellement si besoin.")
    input("Appuie sur Entrée pour démarrer la calibration auto de wrist_flex...")

    print("⚙️ Activation du couple")
    bus.write("Torque_Enable", 1)

    print("🚦 Calibration du moteur wrist_flex...")
    calib = move_to_calibrate(bus, "wrist_flex")

    print("📈 Position de départ :", calib["start_pos"])
    print("📉 Position de fin   :", calib["end_pos"])
    print("🎯 Position zéro     :", calib["zero_pos"])

    # Application d'un petit offset si besoin
    calib = apply_offset(calib, offset=80)
    print("🛠 Offset appliqué. Nouvelle position zéro :", calib["zero_pos"])

    # Déplace le moteur à sa position zéro calibrée
    bus.write("Goal_Position", calib["zero_pos"], motor_names=["wrist_flex"])
    time.sleep(1)

    print("✅ Calibration terminée.")

except Exception as e:
    print(f"\n💥 Erreur : {e}")
finally:
    print("\n🔌 Déconnexion...")
    bus.disconnect()


↪️ Désactivation du couple
✅ Couple OFF. Tu peux bouger le moteur manuellement si besoin.
⚙️ Activation du couple
🚦 Calibration du moteur wrist_flex...
📈 Position de départ : 3249
📉 Position de fin   : 732
🎯 Position zéro     : 1990.5
🛠 Offset appliqué. Nouvelle position zéro : 2070.5
✅ Calibration terminée.

🔌 Déconnexion...


In [ ]:
from lerobot.common.robot_devices.motors.feetech import FeetechMotorsBusConfig, FeetechMotorsBus
import time

# Fonction de test directe du mouvement jusqu'au blocage
def test_move_until_block(bus, motor_name, positive_direction=True):
    count = 0
    while True:
        present_pos = bus.read("Present_Position", motor_name).item()
        present_speed = bus.read("Present_Speed", motor_name).item()
        present_current = bus.read("Present_Current", motor_name).item()

        print(f"📍 Pos: {present_pos} | 🌀 Speed: {present_speed} | ⚡ Current: {present_current}")

        if positive_direction:
            bus.write("Goal_Position", present_pos + 100, motor_name)
        else:
            bus.write("Goal_Position", present_pos - 100, motor_name)

        # Test de condition d'arrêt
        if present_speed == 0 and present_current > 40:
            count += 1
            if count > 5 or present_current > 300:
                print("✅ Blocage détecté.")
                return present_pos
        else:
            count = 0

        time.sleep(0.1)


# === CONFIGURATION ===
config = FeetechMotorsBusConfig(
    port="/dev/ttyACM0",
    motors={"wrist_flex": [4, "sts3215"]}
)

bus = FeetechMotorsBus(config=config)
bus.connect()

try:
    bus.write("Torque_Enable", 1)
    print("🚦 Test move_until_block sur 'wrist_flex'...")

    pos = test_move_until_block(bus, "wrist_flex", positive_direction=False)
    print(f"🔚 Position arrêtée : {pos}")

except Exception as e:
    print(f"💥 Erreur : {e}")
finally:
    print("🔌 Déconnexion...")
    bus.disconnect()


In [ ]:
import json
import os

# Assure-toi d’avoir calib_data depuis ta dernière calibration réussie
from lerobot.common.robot_devices.motors.feetech import FeetechMotorsBusConfig, FeetechMotorsBus
from lerobot.common.robot_devices.robots.feetech_calibration import run_arm_auto_calibration_so100

# Configuration du bus moteur (même que pour la calibration)
config = FeetechMotorsBusConfig(
    port="/dev/ttyACM0",
    motors={
        "shoulder_pan": [1, "sts3215"],
        "shoulder_lift": [2, "sts3215"],
        "elbow_flex": [3, "sts3215"],
        "wrist_flex": [4, "sts3215"],
        "wrist_roll": [5, "sts3215"],
        "gripper": [6, "sts3215"],
    }
)

# Connexion
bus = FeetechMotorsBus(config=config)
bus.connect()

# Tu peux aussi recharger ton ancienne calibration si tu as déjà le calib_data
print("⚙️ Récupération des données de calibration en cours...")
calib_data = run_arm_auto_calibration_so100(
    arm=bus,
    robot_type="so100",
    arm_name="main",
    arm_type="follower"
)

# Enregistrement
save_path = ".cache/calibration/so100/main_follower.json"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
with open(save_path, "w") as f:
    json.dump(calib_data, f, indent=2)

print(f"\n✅ Fichier de calibration enregistré dans : {save_path}")

# Déconnexion
bus.disconnect()